# Gradient  
使用with + GradientTape方法產生梯度向量，並且使用梯度向量物件內的gradient方法實作偏微分

In [40]:
w = tf.Variable(tf.constant([2.0]))

with tf.GradientTape() as t:
    loss = tf.math.exp(w)
#     loss = w * w
grad = t.gradient(loss,w)
print(grad)

#grad = t.gradient(loss,w)

tf.Tensor([7.389056], shape=(1,), dtype=float32)


In [39]:
a = tf.Variable(tf.constant([2.0]))
b = tf.Variable(tf.constant([3.0]))


with tf.GradientTape() as t:
    loss = 2*a**2 + 3*b**3
grad = t.gradient(loss,[a,b])
for i in grad:
    print(i)

tf.Tensor([8.], shape=(1,), dtype=float32)
tf.Tensor([81.], shape=(1,), dtype=float32)


In [41]:
w = tf.Variable(tf.constant([2.0]))

with tf.GradientTape() as t:
    loss = 4**w
grad = t.gradient(loss,w)
print(grad)

tf.Tensor([22.18071], shape=(1,), dtype=float32)


In [42]:
np.log(4)*4**2

22.18070977791825

In [43]:
w = tf.Variable(tf.constant([2.0]))

# 加了persistent=True才能重複使用這個t物件，否則用完即丟
with tf.GradientTape(persistent=True) as t:
    loss = w * w
grad = t.gradient(loss,w)
print(grad)
grad = t.gradient(loss,w)
print(grad)
del t

tf.Tensor([4.], shape=(1,), dtype=float32)
tf.Tensor([4.], shape=(1,), dtype=float32)


In [44]:
w = tf.constant([2.0])

with tf.GradientTape() as t:
    t.watch(w) # 非tf.variable的話就要加上 watch才能微分
    loss = w * w
grad = t.gradient(loss,w)
print(grad)

#grad = t.gradient(loss,w)

tf.Tensor([4.], shape=(1,), dtype=float32)


# Optimization - method 1
兩種方法:
- 共同處:
1. 使用keras的optimizers定義優化方法
2. for迴圈設定調整參數次數
- 不同處:
1. keras內的minimize方法進行疊代
2. processed_grads以list儲存偏微分的項目 + apply_gradients方法對梯度和x的pair值(zip)實作梯度下降

In [8]:
import tensorflow as tf

def fu_minimzie():
    return 2*x*x + x + 3 

x = tf.Variable(1.0) 

opt = tf.keras.optimizers.SGD(learning_rate=0.01)
#opt = tf.keras.optimizers.Adagrad(learning_rate=0.01)
#opt = tf.keras.optimizers.Adam(learning_rate=0.01)
#opt = tf.keras.optimizers.RMSprop(learning_rate=0.01)

# 把x調整1000次
for i in range(1000):
    opt.minimize(fu_minimzie, var_list=[x])
print('x: {}'.format(x.numpy()))

x: -0.24999982118606567


# Optimization - method 2

In [46]:
x = tf.Variable(1.0)

opt = tf.keras.optimizers.SGD(learning_rate=0.1)
for i in range(1000):

    with tf.GradientTape() as tape:
        y = 2*x*x + x + 3 
    grads = tape.gradient(y, [x])
    processed_grads = [g for g in grads]
    grads_and_vars = zip(processed_grads, [x])
#    print('y = {}, x = {}, grads = {} '.format(y.numpy(), x.numpy(), grads[0].numpy()))
    
    opt.apply_gradients(grads_and_vars)
print('x: {}'.format(x.numpy()))
print('y: {}'.format(y.numpy()))


x: -0.2499999850988388
y: 2.875


In [47]:
a = tf.Variable(1.0) 
b = tf.Variable(1.0) 

opt = tf.keras.optimizers.SGD(learning_rate=0.1)
for i in range(1000):
    
    # 多變數調參
    with tf.GradientTape() as tape:
        z = a*a + 2*b*b 
    grads = tape.gradient(z, [a, b])
    processed_grads = [g for g in grads]
    grads_and_vars = zip(processed_grads, [a,b])
#    print('y = {}, x = {}, grads = {} '.format(y.numpy(), x.numpy(), grads[0].numpy()))
    
    opt.apply_gradients(grads_and_vars)
print('a: {}'.format(a.numpy()))
print('b: {}'.format(b.numpy()))

#print('y: {}'.format(y.numpy()))


a: 4.89378601604716e-38
b: 1.930686042851918e-38
